In [ ]:
import random
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver
driver = webdriver.Chrome()  
#Use webdriver.Firefox() for Firefox

# Create a CSV file to store the profile data
csv_file = open("linkedin_profiles.csv", "w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Name", "Job Title"])

def validate_field(field):
    if field:
        return field
    else:
        return 'No results'

# Open LinkedIn
driver.get("https://www.linkedin.com")

# Use an explicit wait to wait for the login form elements to become visible
wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed

# Find the login form elements and enter your credentials
username = ""  # Replace with your LinkedIn email/username
password = ""  # Replace with your LinkedIn password

# Wait for the username field to become visible
username_field = wait.until(EC.visibility_of_element_located((By.ID, "session_key")))
username_field.send_keys(username)

# Wait for the password field to become visible
password_field = wait.until(EC.visibility_of_element_located((By.ID, "session_password")))
password_field.send_keys(password)

password_field.send_keys(Keys.RETURN)

# Wait for the home page to load
wait.until(EC.title_contains("LinkedIn"))

# Find the search bar element and enter the user's name
search_box = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')
user_name = "Mahesh Sharma"  # Replace with the user's name you want to search for
search_box.send_keys(user_name)
search_box.send_keys(Keys.RETURN)

# Wait for the search results page to load
wait.until(EC.title_contains("Search | LinkedIn"))

# Wait for the search results to load completely
time.sleep(3)  # You can adjust the sleep time if needed

# Click on the "People" filter to view only people with the name "David John"
people_filter = driver.find_element(By.XPATH, '//button[text()="People"]')
people_filter.click()

# Wait for the people search results to load
wait.until(EC.presence_of_element_located((By.XPATH, '//li[@class="reusable-search__result-container"]')))

# Scrape the top 10 profiles
for i in range(10):
    try:
        # Define the XPath for the i-th profile result (index starts at 1)
        profile_xpath = f'//li[@class="reusable-search__result-container"][{i + 1}]'

        # Find and click on the profile
        profile_element = driver.find_element(By.XPATH, profile_xpath)
        profile_element.click()

        # Wait for the profile page to load
        wait.until(EC.title_contains("LinkedIn"))

        # Scrape the name, job title, and location using the new XPaths
        try:
            # Locate the elements containing the name, job title, and location using their XPaths
            name_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//h1[normalize-space()="Mahesh Sharma"]')))
            job_title_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="text-body-medium break-words"]')))
            location_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="text-body-small inline t-black--light break-words"]')))

            # Extract text from the elements
            name = validate_field(name_element.text)
            job_title = validate_field(job_title_element.text)
            location = validate_field(location_element.text)

            # Print the scraped information
            print(f"Profile {i + 1} - Name: {name}, Job Title: {job_title}, Location: {location}")

            # Write the data to the CSV file
            wait.until(EC.title_contains("LinkedIn"))
            
            csv_writer.writerow([name, job_title, location])
        except Exception as e:
            print(f"Unable to scrape profile {i + 1} information: {e}")

        # Go back to the search results page
        driver.back()
        wait.until(EC.title_contains("LinkedIn"))

    except Exception as e:
        print(f"Unable to access profile {i + 1}: {e}")

# Close the CSV file
csv_file.close()

# Close the browser
driver.quit()